# CNN on GPU: Cat vs Dog Classifier
> A lightweight yet GPU-intensive demo to show off parallelism, filters, and fast training

## Goal
Train a Convolutional Neural Network (CNN) to classify images of cats vs dogs using GPU acceleration. This helps showcase:
- The parallel nature of convolutions.
- How GPUs drastically reduce training time.
- Visual understanding of filters and feature maps.

## Dataset
Use TensorFlow’s *cats_vs_dogs* dataset:

In [1]:
import tensorflow_datasets as tfds
(train_ds, val_ds), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True,
)

ModuleNotFoundError: No module named 'tensorflow_datasets'

## Preprocessing
Resize and normalize for better training:

In [ ]:
import tensorflow as tf

IMG_SIZE = 128

def format_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    return image, label

train = train_ds.map(format_image).batch(32).prefetch(1)
val = val_ds.map(format_image).batch(32).prefetch(1)